XOR tensorboard/eager

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

print(tf.__version__)

In [ ]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

plt.scatter(x_data[0][0],x_data[0][1], c='red' , marker='^')
plt.scatter(x_data[3][0],x_data[3][1], c='red' , marker='^')
plt.scatter(x_data[1][0],x_data[1][1], c='blue' , marker='^')
plt.scatter(x_data[2][0],x_data[2][1], c='blue' , marker='^')

plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    return features, labels

log_path = "/Users/johnk/logs/"
writer = tf.summary.create_file_writer(log_path)

pip install tensorboard
tensorboard --logdir=./logs/

In [ ]:
W1 = tf.Variable(tf.random.normal((2, 10)), name='weight1')
b1 = tf.Variable(tf.random.normal((10,)), name='bias1')

W2 = tf.Variable(tf.random.normal((10, 10)), name='weight2')
b2 = tf.Variable(tf.random.normal((10,)), name='bias2')

W3 = tf.Variable(tf.random.normal((10, 10)), name='weight3')
b3 = tf.Variable(tf.random.normal((10,)), name='bias3')

W4 = tf.Variable(tf.random.normal((10, 1)), name='weight4')
b4 = tf.Variable(tf.random.normal((1,)), name='bias4')

def neural_net(features, step):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
    hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

    with tf.contrib.summary.record_summaries_every_n_global_steps(1):
        tf.contrib.summary.histogram("weights1", W1, step=step)
        tf.contrib.summary.histogram("biases1", b1, step=step)
        tf.contrib.summary.histogram("layer1", layer1, step=step)

        tf.contrib.summary.histogram("weights2", W2, step=step)
        tf.contrib.summary.histogram("biases2", b2, step=step)
        tf.contrib.summary.histogram("layer2", layer2, step=step)

        tf.contrib.summary.histogram("weights3", W3, step=step)
        tf.contrib.summary.histogram("biases3", b3, step=step)
        tf.contrib.summary.histogram("layer3", layer3, step=step)

        tf.contrib.summary.histogram("weights4", W4, step=step)
        tf.contrib.summary.histogram("biases4", b4, step=step)
        tf.contrib.summary.histogram("hypothesis", hypothesis, step=step)
    return hypothesis

tf.summary.histogram(name, data, step=None ...) histogram 형태의 로그를 저장한다.

with writer.as_default():  epoch 별 손실함수 값이 파일에 저장되며, 이를 불러와 시각화 된 형태로 볼 수 있게 된다.

tf.summary.scalar : scalar값 형태의 로그 저장

In [ ]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    with writer.as_default():
        tf.summary.scalar('loss', cost, step=step)
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(hypothesis, features, labels, step):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features, step),labels)
    return tape.gradient(loss_value, [W1, W2, W3, W4, b1, b2, b3, b4])

In [ ]:
EPOCHS = 3000

for step in range(EPOCHS):    
    for features, labels  in dataset:
        features, labels = preprocess_data(features, labels)
        grads = grad(neural_net(features, step), features, labels, step)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, W4, b1, b2, b3, b4]))
        if step % 50 == 0:
            loss_value = loss_fn(neural_net(features, step),labels)
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data, step),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Tensorboard 실행하기

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/xor

매 학습마다의 loss 값과 각종 모델 값들을 확인 할 수 있다